In [4]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import json
import re
import csv
import pandas as pd
from collections import Counter
from tqdm import tqdm

Создаем словарь идиома - ссылка на её страницу в wiktionary.

In [5]:
url = 'https://en.wiktionary.org/wiki/Category:French_multiword_terms'
dict_of_links = {}
while True:
    page = requests.get(url)
    html = page.text
    soup = BeautifulSoup(html)
    for item in soup.select("div[class='mw-category-generated'] a[href^='/wiki/']"):
        dict_of_links[re.search('(?<=>)(.*)(?=<)', str(item)).group(0)] = \
        'https://en.wiktionary.org'+re.search('(?<=href=")([^"]*)', str(item)).group(0)
    next_page_available = 0
    for i in soup.find_all('a'):
        if i.string == 'next page':
            url='https://en.wiktionary.org/'+i['href']
            next_page_available = 1
            break
    if not next_page_available:
        break
len(dict_of_links)

16441

Нам нужны будут определения идиом, для их извлечения с интернет-страницы воспользуемся пакетом WiktionaryParser. Загрузим его:

In [6]:
from wiktionaryparser import WiktionaryParser
parser = WiktionaryParser()

У многих идиом на странице нет их определения, а есть ссылка на альтернативную форму или иное написание. Нам нужно углубляться по ссылке до тех пор, пока мы не попадем на определение. Сделаем функцию для доставания финального определения:

In [ ]:
# def final_definition(collocation):
#     check = 1
#     while check == True:
#         collocation_info = parser.fetch(collocation, 'french')
#         if collocation_info==[]:
#             return ' '
#         else:

#                 if len(collocation_info[0]['definitions'][0]['text'])>1:
#                     for definition in collocation_info[0]['definitions'][0]['text'][1:]:
#                         if "alternative form of " in definition.lower():
#                             collocation = re.search('(?<=[Aa]lternative form of )(.*)', definition).group(0)
#                         elif "alternative spelling of " in definition.lower():
#                             collocation = re.search('(?<=[Aa]lternative spelling of )(.*)', definition).group(0)
#                         elif "synonym of " in definition.lower():
#                             collocation = re.search('(?<=[Ss]ynonym of )(.*)', definition).group(0)
#                         else:
#                             check = 0
#                 else:
#                     check = 0     
#             else:
#                 check = 0
#     collocation_info = parser.fetch(collocation, 'french')
#     if collocation_info!=[]:
#         if collocation_info[0]['definitions']!=[]:
#             if len(collocation_info[0]['definitions'][0]['text'])>1:
#                 return collocation_info[0]['definitions'][0]['text'][1:]
#             else:
#                 return ' '
#         else:
#             return ' '

In [79]:
def final_definition(collocation):
    check = 1
    while check == True:
        collocation_info = parser.fetch(collocation, 'french')
        if collocation_info==[]:
            return ' '
        else:
            if collocation_info[0]['definitions']==[]:
                return ' '
            else:
                if len(collocation_info[0]['definitions'][0]['text'])<2:
                    return ' '
                else:
                    for definition in collocation_info[0]['definitions'][0]['text'][1:]:
                        if "alternative form of " in definition.lower():
                            collocation = re.search('(?<=[Aa]lternative form of )([^,]*)', definition).group(0)
                        elif "alternative spelling of " in definition.lower():
                            collocation = re.search('(?<=[Aa]lternative spelling of )([^,]*)', definition).group(0)
                        elif "synonym of " in definition.lower():
                            collocation = re.search('(?<=[Ss]ynonym of )([^,]*)', definition).group(0)
                        else:
                            check = 0  
            
    collocation_info = parser.fetch(collocation, 'french')
    if collocation_info==[]:
        return ' '
    else:
        if collocation_info[0]['definitions']==[]:
            return ' '
        else:
            if len(collocation_info[0]['definitions'][0]['text'])>1:
                return collocation_info[0]['definitions'][0]['text'][1:]
            else:
                return ' '

In [78]:
final_definition('à le')

' '

Достаём определения и помещаем в список:  
1. часть речи, соответствующую идиоме  
2. строку, содержащую все определения через точку с запятой  
3. ссылку на страницу идиомы  

Этот список добавляем в список списков.

In [19]:
# del dict_of_links['24 Sussex']
# del dict_of_links['3 Amigos']
# del dict_of_links['à le']

In [80]:
import time

list_for_file = []
for collocation in tqdm(dict_of_links):
    timeout = 1   # [seconds]
    timeout_start = time.time()
    while time.time() < timeout_start + timeout:
        try:
            collocation_info = parser.fetch(collocation, 'french')
            if collocation_info!=[]:
                if collocation_info[0]['definitions']==[]:
                    break
                else:
    #                 добавляем идиому:
                    list_for_collocation = [collocation]
#     берем только словосочетания, которые имеют идиоматическое значение
                    if "idiom" not in "; ".join(final_definition(collocation)):
                        break
                    else:
 #         добавляем определения, предварительно углубившись до нужной страницы
                        list_for_collocation.append("; ".join(final_definition(collocation)))
        #     добавляем ссылку на страницу идиомы:
                        list_for_collocation.append(dict_of_links[collocation])
        #                 добавляем literal meaning, если оно есть
                        if collocation_info[0]['etymology']!=[]:
                            if not re.search('(?<=Literally, ")([^"]*)', collocation_info[0]['etymology']):
                                list_for_collocation.append('')
                            else:
                                list_for_collocation.append(re.search('(?<=Literally, ")([^"]*)', \
                                                                      collocation_info[0]['etymology']).group(0))
                        else:
                            list_for_collocation.append('')
        #     добавляем часть речи, если есть:
                        if 'partOfSpeech' in collocation_info[0]['definitions'][0]:
                            list_for_collocation.append(collocation_info[0]['definitions'][0]['partOfSpeech'])
                        else:
                            list_for_collocation.append('')
                        list_for_file.append(list_for_collocation)
        except AttributeError:
            pass

100%|███████████████████████████████████| 16438/16438 [6:33:06<00:00,  1.43s/it]


In [81]:
len(list_for_file)

232

In [82]:
list_for_file[:2]

[['à bras ouverts',
  '(idiomatic) with open arms',
  'https://en.wiktionary.org/wiki/%C3%A0_bras_ouverts',
  '',
  'adverb'],
 ['à contre-courant',
  'upstream; (idiomatic) against the grain (contrary to what is expected.)',
  'https://en.wiktionary.org/wiki/%C3%A0_contre-courant',
  '',
  'adverb']]

Записываем в файл:

In [83]:
with open('fr_idioms_1701.tsv', 'w', newline='') as record_file:
    tsv_writer = csv.writer(record_file, delimiter='\t', lineterminator='\n')
    tsv_writer.writerow(['Collocation', 'Part_of_speech', 'Literal meaning', 'Definition', 'Link'])
    for collocation_info in list_for_file:
        tsv_writer.writerow([collocation_info[0], collocation_info[4], collocation_info[3],\
                             collocation_info[1], collocation_info[2]])

Посмотрим, что получилось:

In [129]:
df = pd.read_csv("fr_idioms_1701.tsv",sep='\t')

In [85]:
def left_align(df):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )
    return left_aligned_df

In [86]:
pd.options.display.max_colwidth=100
pd.options.display.max_rows=500
# display(left_align(df[['Collocation','Part_of_speech','Definition']].head(500)))
display(left_align(df.head(50)))

,Collocation,Part_of_speech,Literal meaning,Definition,Link
0,à bras ouverts,adverb,nan,(idiomatic) with open arms,https://en.wiktionary.org/wiki/%C3%A0_bras_ouverts
1,à contre-courant,adverb,nan,upstream; (idiomatic) against the grain (contrary to what is expected.),https://en.wiktionary.org/wiki/%C3%A0_contre-courant
2,à contre-jour,adverb,nan,backlit; (idiomatic) in the darkness; in the dark,https://en.wiktionary.org/wiki/%C3%A0_contre-jour
3,à grande échelle,adverb,nan,(idiomatic) on a large scale,https://en.wiktionary.org/wiki/%C3%A0_grande_%C3%A9chelle
4,à la,preposition,nan,"Used other than figuratively or idiomatically: see à,‎ la.; a la, in the style or manner of (with a feminine singular adjective or a proper noun)",https://en.wiktionary.org/wiki/%C3%A0_la
5,à la bonne franquette,adverb,nan,"(idiomatic, cooking) in a homely manner, without ceremony, informally",https://en.wiktionary.org/wiki/%C3%A0_la_bonne_franquette
6,à la clé,adverb,nan,"(idiomatic) as a reward, at stake, up for grabs",https://en.wiktionary.org/wiki/%C3%A0_la_cl%C3%A9
7,à la grecque,adverb,nan,"Used other than figuratively or idiomatically: see à la,‎ grec.; (cooking) cooked in olive oil and served cold (Can we verify(+) this sense?)",https://en.wiktionary.org/wiki/%C3%A0_la_grecque
8,à la ronde,adverb,nan,"(idiomatic, colloquial) around",https://en.wiktionary.org/wiki/%C3%A0_la_ronde
9,à la Saint-Glinglin,adverb,nan,"(humorous, idiomatic, sarcastic) when pigs fly",https://en.wiktionary.org/wiki/%C3%A0_la_Saint-Glinglin


In [87]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# print(list(chunks(my_list, 3)))

In [97]:
url = 'https://en.wiktionary.org/wiki/Appendix:French_idioms'
list_of_collocations = []

page = requests.get(url)
html = page.text
soup = BeautifulSoup(html)
list_of_lists = []
counting = 0
for i in soup.select("tr td"):
    list_of_lists.append(i)
list_of_lists_1 = list_of_lists[1:]
list_of_lists_2 = list(chunks(list_of_lists_1, 3))
for each_list in list_of_lists_2:
    if each_list[0].find('a'):
        list_of_collocations.append([each_list[0]('a')[0].string,\
                                     each_list[1].text.split('\n')[0],\
                                     each_list[2].text.split('\n')[0]])
    else:
        list_of_collocations.append([each_list[0].string,\
                                     each_list[1].text.split('\n')[0],\
                                     each_list[2].text.split('\n')[0]])
list_of_collocations[:2]

[['à bon chat, bon rat', 'tit-for-tat', 'to a good cat, a good rat'],
 ['à chaque jour suffit sa peine',
  'each day as it comes',
  "each day's pain is sufficient for it (reference to the Biblical verse Matthew 6:34)"]]

In [89]:
with open('fr_idioms_for_sure_1701.tsv', 'w', newline='') as record_file:
    tsv_writer = csv.writer(record_file, delimiter='\t', lineterminator='\n')
    tsv_writer.writerow(['Collocation', 'Meaning', 'Literal Meaning'])
    for collocation in list_of_collocations:
        tsv_writer.writerow(collocation)

In [96]:
df_sure = pd.read_csv("fr_idioms_for_sure.tsv",sep='\t')
df_sure.head()

,Idiom,Meaning,Literal Meaning
0,"à bon chat, bon rat",tit-for-tat,"to a good cat, a good rat"
1,à chaque jour suffit sa peine,each day as it comes,each day's pain is sufficient for it (reference to the Biblical verse Matthew 6:34)
2,à cheval donné on ne regarde pas les dents,"don't criticize gifts, accept them gratefully",Don't look a gift horse in the mouth.
3,amoureuse comme une chatte,very affectionate,in love like a female cat
4,appeler un chat un chat,to tell it like it is; to call a spade a spade,to call a cat a cat


In [ ]:
url = 'https://en.wiktionary.org/wiki/Appendix:French_idioms'
dict_of_links_idioms = {}
while True:
    page = requests.get(url)
    html = page.text
    soup = BeautifulSoup(html)
    for item in soup.select("tr td a[href^='/wiki/']"):
        dict_of_links_idioms[re.search('(?<=>)(.*)(?=<)', str(item)).group(0)] = \
        'https://en.wiktionary.org'+re.search('(?<=href=")([^"]*)', str(item)).group(0)
    next_page_available = 0
    for i in soup.find_all('a'):
        if i.string == 'next page':
            url='https://en.wiktionary.org/'+i['href']
            next_page_available = 1
            break
    if not next_page_available:
        break
# dict_of_links_idioms

In [115]:
part_of_speech = []
for collocation in tqdm(df_sure['Idiom']):
    try:
        collocation_info = parser.fetch(collocation, 'french')
        if collocation_info!=[]:
            if collocation_info[0]['definitions']!=[]:
                if 'partOfSpeech' in collocation_info[0]['definitions'][0]:
                    part_of_speech.append(collocation_info[0]['definitions'][0]['partOfSpeech'])
                else:
                    part_of_speech.append(None)
            else:
                part_of_speech.append(None)
        else:
            part_of_speech.append(None)
    except AttributeError:
        part_of_speech.append(None)
        pass

100%|█████████████████████████████████████████| 210/210 [01:30<00:00,  2.31it/s]


In [116]:
print(len(df_sure['Idiom']))
print(len(part_of_spceech))

210
210


In [124]:
df_sure['Part_of_speech'] = part_of_speech

In [125]:
links = []
for collocation in tqdm(df_sure['Idiom']):
    if collocation in dict_of_links_idioms:
        links.append(dict_of_links_idioms[collocation])
    else:
        links.append(None)
len(links)

100%|█████████████████████████████████████| 210/210 [00:00<00:00, 279975.79it/s]


210

In [126]:
df_sure['Link'] = links

In [127]:
df_sure.head()

,Idiom,Meaning,Literal Meaning,Part_of_speech,Link
0,"à bon chat, bon rat",tit-for-tat,"to a good cat, a good rat",proverb,"https://en.wiktionary.org/wiki/%C3%A0_bon_chat,_bon_rat"
1,à chaque jour suffit sa peine,each day as it comes,each day's pain is sufficient for it (reference to the Biblical verse Matthew 6:34),proverb,https://en.wiktionary.org/wiki/%C3%A0_chaque_jour_suffit_sa_peine
2,à cheval donné on ne regarde pas les dents,"don't criticize gifts, accept them gratefully",Don't look a gift horse in the mouth.,proverb,https://en.wiktionary.org/wiki/%C3%A0_cheval_donn%C3%A9_on_ne_regarde_pas_les_dents
3,amoureuse comme une chatte,very affectionate,in love like a female cat,None,None
4,appeler un chat un chat,to tell it like it is; to call a spade a spade,to call a cat a cat,verb,https://en.wiktionary.org/wiki/appeler_un_chat_un_chat


In [144]:
df_sure = df_sure.rename(columns={'Idiom': 'Collocation', 'Meaning': 'Definition', 'Literal Meaning': \
                                 'Literal meaning'})

In [145]:
df_sure = df_sure[['Collocation', 'Part_of_speech', 'Literal meaning', 'Definition', 'Link']]

In [141]:
df.head()

,Collocation,Part_of_speech,Literal meaning,Definition,Link
0,à bras ouverts,adverb,NaN,(idiomatic) with open arms,https://en.wiktionary.org/wiki/%C3%A0_bras_ouverts
1,à contre-courant,adverb,NaN,upstream; (idiomatic) against the grain (contrary to what is expected.),https://en.wiktionary.org/wiki/%C3%A0_contre-courant
2,à contre-jour,adverb,NaN,backlit; (idiomatic) in the darkness; in the dark,https://en.wiktionary.org/wiki/%C3%A0_contre-jour
3,à grande échelle,adverb,NaN,(idiomatic) on a large scale,https://en.wiktionary.org/wiki/%C3%A0_grande_%C3%A9chelle
4,à la,preposition,NaN,"Used other than figuratively or idiomatically: see à,‎ la.; a la, in the style or manner of (wit...",https://en.wiktionary.org/wiki/%C3%A0_la


In [146]:
df_sure.head()

,Collocation,Part_of_speech,Literal meaning,Definition,Link
0,"à bon chat, bon rat",proverb,"to a good cat, a good rat",tit-for-tat,"https://en.wiktionary.org/wiki/%C3%A0_bon_chat,_bon_rat"
1,à chaque jour suffit sa peine,proverb,each day's pain is sufficient for it (reference to the Biblical verse Matthew 6:34),each day as it comes,https://en.wiktionary.org/wiki/%C3%A0_chaque_jour_suffit_sa_peine
2,à cheval donné on ne regarde pas les dents,proverb,Don't look a gift horse in the mouth.,"don't criticize gifts, accept them gratefully",https://en.wiktionary.org/wiki/%C3%A0_cheval_donn%C3%A9_on_ne_regarde_pas_les_dents
3,amoureuse comme une chatte,None,in love like a female cat,very affectionate,None
4,appeler un chat un chat,verb,to call a cat a cat,to tell it like it is; to call a spade a spade,https://en.wiktionary.org/wiki/appeler_un_chat_un_chat


In [147]:
french_idioms_altogether_1701 = pd.concat([df,df_sure]).drop_duplicates('Collocation').reset_index(drop=True)
french_idioms_altogether_1701.tail()

,Collocation,Part_of_speech,Literal meaning,Definition,Link
423,vaches maigres,noun,thin cows (reference to the Biblical chapter of Genesis 41),lean times,https://en.wiktionary.org/wiki/vaches_maigres
424,vendre la peau de l'ours avant de l'avoir tué,proverb,to sell the bear's skin before having killed it,to count your chickens before they hatch,https://en.wiktionary.org/wiki/vendre_la_peau_de_l%27ours_avant_de_l%27avoir_tu%C3%A9
425,vieux bouc,None,old billy goat,"dirty old man, old pervert",None
426,vieux renard,None,old fox,"sly fox, clever as a fox",None
427,voler de ses propres ailes,verb,to fly off using one's own wings,to stand on one's own two feet,https://en.wiktionary.org/wiki/voler_de_ses_propres_ailes


In [153]:
french_idioms_altogether_1701 = french_idioms_altogether_1701[french_idioms_altogether_1701.Part_of_speech != \
                                                              'proverb']
french_idioms_altogether_1701.reset_index(drop=True, inplace=True)
french_idioms_altogether_1701.tail()

,Collocation,Part_of_speech,Literal meaning,Definition,Link
410,vaches grasses,None,fat cows (reference to the Biblical chapter of Genesis 41),prosperous years,None
411,vaches maigres,noun,thin cows (reference to the Biblical chapter of Genesis 41),lean times,https://en.wiktionary.org/wiki/vaches_maigres
412,vieux bouc,None,old billy goat,"dirty old man, old pervert",None
413,vieux renard,None,old fox,"sly fox, clever as a fox",None
414,voler de ses propres ailes,verb,to fly off using one's own wings,to stand on one's own two feet,https://en.wiktionary.org/wiki/voler_de_ses_propres_ailes


In [154]:
french_idioms_altogether_1701.to_csv('french_idioms_altogether_1701', sep='\t')